In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\mnist\3.0.1.incomplete9ZZRGW\mnist-train.tfrecord*...:   0%|          | 0/6000…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\mnist\3.0.1.incomplete9ZZRGW\mnist-test.tfrecord*...:   0%|          | 0/10000…

Dataset mnist downloaded and prepared to ~\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [11]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image,label):
    image = tf.cast(image, tf.float32)
    image/=255
    return image,label

scaled_train_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

buffer_size = 10000

shuffled_data = scaled_train_validation_data.shuffle(buffer_size)

validation_data = shuffled_data.take(num_validation_samples)
train_data = shuffled_data.skip(num_validation_samples)

batch_size = 100

train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [22]:
input_size = 784
output_size = 10
hidden_layer_size = 150

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')

])

In [23]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(train_data, epochs=5, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 2s - loss: 0.2929 - accuracy: 0.9149 - val_loss: 0.1239 - val_accuracy: 0.9637 - 2s/epoch - 3ms/step
Epoch 2/5
540/540 - 1s - loss: 0.1090 - accuracy: 0.9670 - val_loss: 0.0907 - val_accuracy: 0.9708 - 1s/epoch - 2ms/step
Epoch 3/5
540/540 - 1s - loss: 0.0729 - accuracy: 0.9775 - val_loss: 0.0674 - val_accuracy: 0.9807 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.0566 - accuracy: 0.9818 - val_loss: 0.0759 - val_accuracy: 0.9785 - 1s/epoch - 3ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0451 - accuracy: 0.9855 - val_loss: 0.0579 - val_accuracy: 0.9830 - 1s/epoch - 3ms/step


In [25]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 376ms/step - loss: 0.0887 - accuracy: 0.9759
